In [2]:
%matplotlib inline

# Calculate electrochemical equilbria

In [3]:
from scipy import optimize

import fipy as fp

In [4]:
Vs = fp.Variable("0.018 l/mol")
R = fp.Variable("1 Nav * kB")
T = fp.Variable("298 K")
Faraday = fp.Variable("1 Nav * e")

simple mesh that only has a cell for electrode and a cell for the electrolyte

In [5]:
mesh = fp.Grid1D(nx=2)

In [6]:
def solid(X, interstitials, substitutionals, solvent):
    """solve for electron and cation concentration, given impurity salt and solvent
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[0].value
        
    charge = 0. + X[0] * interstitials[0].z + X[1] * substitutionals[0].z
    for Xj in components:
        charge += Xj.z * Xj[0].value

    return [subs, charge]

In [7]:
def liquid(X, interstitials, substitutionals, solvent):
    """solve for solvent and anion concentration, given cation(s) and impurity electron
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[1].value
        
    charge = 0. + X[0] * solvent.z + X[1] * substitutionals[1].z
    for Xj in components:
        charge += Xj.z * Xj[1].value

    return [subs, charge]

## monovalent cation and salt

### $X_\mathrm{impurity} = 10^{-6}$

In [9]:
impurity = 1e-6

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+$", value=[0., impurity]),
    fp.CellVariable(mesh=mesh, name="$A^-$", value=[impurity, 0]),
    fp.CellVariable(mesh=mesh, name="$B^+$", value=[impurity, (Vs * "1 mol/l").inBaseUnits()])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +1
substitutionals[1].z = -1
substitutionals[2].z = +1
N.z                  = 0

components = interstitials + substitutionals + [N]

In [11]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

frac = 1.
charge = 0.
for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj
    frac -= Xj
    charge += Xj * Xj.z
print frac
print charge

-13.1223603774 $e^-$ [  4.99998500e-01   1.00000000e-06]
-13.1223603774 $M^+$ [  4.99998500e-01   1.00000000e-06]
9.79812703688 $A^-$ [  1.00000000e-06   1.80000000e-02]
9.79812703688 $B^+$ [  1.00000000e-06   1.80000000e-02]
13.7788444989 $N$ [  1.00000000e-06   9.63998000e-01]
[ -2.47549614e-17  -1.11022302e-16]
[ 0.  0.]


## $X_\mathrm{impurity} = 10^{-9}$

In [12]:
impurity = 1e-9

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+$", value=[0., impurity]),
    fp.CellVariable(mesh=mesh, name="$A^-$", value=[impurity, 0]),
    fp.CellVariable(mesh=mesh, name="$B^+$", value=[impurity, (Vs * "1 mol/l").inBaseUnits()])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +1
substitutionals[1].z = -1
substitutionals[2].z = +1
N.z                  = 0

components = interstitials + substitutionals + [N]

In [13]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

frac = 1.
charge = 0.
for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj
    frac -= Xj
    charge += Xj * Xj.z
print frac
print charge

-20.0301186534 $e^-$ [  4.99999998e-01   1.00000000e-09]
-20.0301186534 $M^+$ [  4.99999998e-01   1.00000000e-09]
16.7058823159 $A^-$ [  1.00000000e-09   1.80000000e-02]
16.7058823159 $B^+$ [  1.00000000e-09   1.80000000e-02]
20.6866018505 $N$ [  1.00000000e-09   9.63999998e-01]
[  2.61765081e-17   0.00000000e+00]
[ 0.  0.]


## $X_\mathrm{impurity} = 10^{-18}$

In [14]:
impurity = 1e-18

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+$", value=[0., impurity]),
    fp.CellVariable(mesh=mesh, name="$A^-$", value=[impurity, 0]),
    fp.CellVariable(mesh=mesh, name="$B^+$", value=[impurity, (Vs * "1 mol/l").inBaseUnits()])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +1
substitutionals[1].z = -1
substitutionals[2].z = +1
N.z                  = 0

components = interstitials + substitutionals + [N]

In [15]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

frac = 1.
charge = 0.
for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj
    frac -= Xj
    charge += Xj * Xj.z
print frac
print charge

-40.7533844933 $e^-$ [  5.00000000e-01   1.00000000e-18]
-40.7533844933 $M^+$ [  5.00000000e-01   1.00000000e-18]
37.4291481528 $A^-$ [  1.00000000e-18   1.80000000e-02]
37.4291481528 $B^+$ [  1.00000000e-18   1.80000000e-02]
41.4098676895 $N$ [  1.00000000e-18   9.64000000e-01]
[ -3.00000000e-18   0.00000000e+00]
[ 0.  0.]
